In [ ]:
#Tweepy related imports
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API
from tweepy import Cursor

import glob

#Other necessary imports
import numpy as np
import pandas as pd
import datetime
import time
import sys

from os import listdir
from os.path import isfile, join
%run credentials.ipynb

In [ ]:
#Class for using twitter api for a particular user
class TwitterClient():
    
    """
    Class for using the Twitter API
    """
    
    def __init__(self,user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
        self.twitter_user = user 
        
    def get_twitter_client_api(self):
        return self.twitter_client
        
    def get_tweets(self,num_tweets,start_date=datetime.datetime(2000,1,1,0,0,0)):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline,id=self.twitter_user).items(num_tweets):
            if tweet.created_at > start_date:
                tweets.append(tweet)
        return tweets
    
    def get_followers(self):
        follower_list = []
        for page in Cursor(self.twitter_client.followers,id=self.twitter_user).pages():
            follower_list.extend(page)
            time.sleep(120)
        return follower_list
        
    def get_friends(self,num_friends):
        friend_list = []
        for friend in Cursor(self.twitter_client.friends,id=self.twitter_user).items(num_friends):
            friend_list.append(friend)
        return friend_list
    
    def get_tweets_by_location(self,lat,long,num_tweets):
        tweets_loc = []
        for tweet in Cursor(self.twitter_client.search,q="*",geocode=str(lat)+","+str(long)+",1km",lang="en").items(num_tweets):
            tweets_loc.append(tweet)
        return tweets_loc

In [ ]:
class TwitterAuthenticator():
    """
    Class for Authentication to Twitter App
    """
    
    
    def authenticate_twitter_app(self):
        auth = OAuthHandler(CONSUMER_KEY,CONSUMER_KEY_SECRET)
        auth.set_access_token(ACCESS_TOKEN,ACCESS_TOKEN_SECRET)
        return auth
        
    

In [ ]:
class TwitterStreamer():
    """
    Class for streaming live tweets
    """
    
    def __init__(self):
        self.twitter_authenticator = TwitterAuthenticator()
    
    def stream_tweets(self,filename,hashtag_list,locations,count):
        listener = TwitterListener(filename)
        auth = self.twitter_authenticator.authenticate_twitter_app()
        stream = Stream(auth,listener)
        stream.filter(track=hashtag_list,locations=locations,count=count)

In [ ]:
class TwitterListener(StreamListener):
    
    """
    Class for printing tweets
    """
    
    def __init__(self,filename):
        self.fetched_tweets_filename = filename
    
    def on_data(self,data):
        try:
            print(data)
            with open(self.fetched_tweets_filename,'a') as f:
                f.write(data)
            return True
        except BaseException as e:
            print("Error: %s" % str(e))
        return True
    def on_error(self,status):
        if status == 420:
            return False #Because rate limit of Twitter Data
        print(status)
        
    

In [ ]:
class TweetAnalyzer():
    """
    Class for analyzing tweets
    """
    def tweets_to_dataframe(self,tweets):
        #df_tweets = pd.DataFrame(data=[tweet.text for tweet in tweets, tweet.user.name for tweet in tweets],columns=["Tweets","Username"])
        data = {'Tweets':[tweet.text for tweet in tweets], 'Username':[tweet.user.screen_name for tweet in tweets]}
        df = pd.DataFrame(data)
        return df    
    def tweets_to_list(self,users_json):
        users_list = [user.screen_name for user in users_json]
        return users_list
        

In [ ]:

#Few tests on the twitter classes created
twitter_client = TwitterClient()
twitter_analyzer = TweetAnalyzer()
api = twitter_client.get_twitter_client_api()

#tweets_surat = twitter_client.get_tweets_by_location(19.136295, 72.910424,80)
#df = twitter_analyzer.tweets_to_dataframe(tweets_surat)
#print(df)

#first_tweet = df["Tweets"].iloc[0]
#print(first_tweet)

twitter_client_modi = TwitterClient("JainKeval108")
modi_tweets = twitter_client_modi.get_tweets(1)
#print((modi_tweets))
#print(twitter_analyzer.tweets_to_dataframe(modi_tweets))
temp = twitter_analyzer.tweets_to_dataframe(modi_tweets)
print(temp['Tweets'].iloc[0])

RT @kunalkamra88: Lapdog journalist : Congress is to be blamed for fall of Indian economy...

Me : Yup &amp; BJP is responsible for the growth…
